In [7]:
# 필요한 library 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.5 MB/s eta 0:00:0031m14.6 MB/s eta 0:00:01


In [5]:
import pandas as pd
# 파인튜닝 데이터 셋
ori_t = pd.read_excel('감성대화말뭉치(원시데이터)_Training.xlsx')
ori_v = pd.read_excel('감성대화말뭉치(원시데이터)_Validation.xlsx')

# 빈 cell 공백 처리
ori_t = ori_t.fillna("")
ori_v = ori_v.fillna("")

ori_t["사람문장"] = ori_t["사람문장1"].astype(str)+ori_t["사람문장2"].astype(str)+ori_t["사람문장3"].astype(str)
ori_v["사람문장"] = ori_v["사람문장1"].astype(str)+ori_v["사람문장2"].astype(str)+ori_v["사람문장3"].astype(str)

df_concat = pd.concat([ori_t, ori_v])
chatbot = df_concat[["사람문장","감정_대분류"]]

chatbot = chatbot.rename({"감정_대분류": "Emotion"}, axis=1)
chatbot = chatbot.rename({"사람문장": "Sentence"}, axis=1)

chatbot["Emotion"] = chatbot["Emotion"].apply(lambda x:x.strip())

ModuleNotFoundError: No module named 'kobert'

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.model_selection import train_test_split

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [ ]:
# 6개의 감정 class -> 숫자
chatbot.loc[(chatbot['Emotion']=="불안"),'Emotion']= 0 # 불안 => 0
chatbot.loc[(chatbot['Emotion']=="분노"),'Emotion']= 1 # 분노 => 1
chatbot.loc[(chatbot['Emotion']=="상처"),'Emotion']= 2 # 상처 => 2
chatbot.loc[(chatbot['Emotion']=="슬픔"),'Emotion']= 3 # 슬픔 => 3
chatbot.loc[(chatbot['Emotion']=="당황"),'Emotion']= 4 # 당황 => 4
chatbot.loc[(chatbot['Emotion']=="기쁨"),'Emotion']= 5 # 기쁨 => 5

data_list = [ ]
for q, label in zip(chatbot['Sentence'], chatbot['Emotion']):
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

print(data)
print(data_list[:10])


In [ ]:
#train & test 데이터로 나누기
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

# 데이터셋 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [ ]:
# BERTSentenceTransform 클래스 정의
# BERT 모델에 입력으로 넣기 위한 전처리 과정
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset] # 문장 변환
        self.labels = [np.int32(i[label_idx]) for i in dataset] # label 변환

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self): # 전체 데이터셋의 길이 반환
        return (len(self.labels))

In [ ]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-6